In [62]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [66]:
fdp = "https://www.fdpbt.de/fraktion/abgeordnete"
source_fdp = requests.get(fdp).text
soup_fdp = BeautifulSoup(source_fdp, 'html.parser')

cdu = "https://www.cducsu.de/hier-stellt-die-cducsu-bundestagsfraktion-ihre-abgeordneten-vor"
source_cdu = requests.get(cdu).text
soup_cdu = BeautifulSoup(source_cdu, 'html.parser')

spd = "https://www.spdfraktion.de/abgeordnete/alle?wp=19&view=list&old=19"
source_spd = requests.get(spd).text
soup_spd = BeautifulSoup(source_spd, 'html.parser')

# for Die Linke, one needs to extract the twitter info from each individual MdB website
linke_base = "https://www.linksfraktion.de/fraktion/abgeordnete/"
letters = [['a', 'e'], ['f', 'j'], ['k', 'o'], ['p', 't'], ['u', 'z']]
linke_name_bins = []

for letter in letters:
    extension = f'{letter[0]}-bis-{letter[1]}/' 
    linke_name_bins.append(linke_base + extension)

In [67]:
all_abg_fdp = soup_fdp.find(class_ = 'person-list').find_all(class_ = 'person-item-wrapper')
all_abg_cdu = soup_cdu.find_all(class_ = 'teaser delegates')
all_abg_spd = soup_spd.find_all(class_ = 'views-row')
 
all_abg_linke = []
for name_bin in linke_name_bins:
    source = requests.get(name_bin).text
    soup = BeautifulSoup(source, 'html.parser')
    for abg in soup.find_all('div', attrs = {'class': 'col-xs-12 col-sm-12 col-md-6 col-lg-6'}):
        extension = abg.find('h2').find('a')['href'].lstrip('/fraktion/abgeordnete/')
        all_abg_linke.append(linke_base + extension)

In [69]:
twitter_list = []
for abg in all_abg_fdp:
    name_field = abg.find(class_ = 'person-name')
    funktion = name_field.find('span').text.strip()
    name = name_field.text.strip('\n').strip().strip(funktion).strip('\n').strip()
    twitter = abg.find('a', attrs = {'class': 'tw'}, href = True)
    twitter_list.append(
        {
        'partei': "FDP",
        'name': name,
        'twitter': twitter['href'] if twitter is not None else ""
            }
        )
    
for abg in all_abg_cdu:
    twitter = abg.find(class_ = 'twitter')
    twitter_list.append(
        {
        'partei': "CDU/CSU",
        'name': abg.find('h2').find('span').text,
        'twitter': twitter.find('a', href = True)['href'] if twitter is not None else ""
            }
        )
    
for abg in all_abg_spd:
    twitter = abg.find(class_ = 'ico_twitter')
    twitter_list.append(
        {
        'partei': "SPD",
        'name': abg.find('h3').find('a').get_text(),
        'twitter': twitter['href'] if twitter is not None else ""
            }
        )
    
for abg in linke_abg:
    abg_source = requests.get(abg).text
    abg_soup = BeautifulSoup(abg_source, 'html.parser')
    twitter = abg_soup.find('a', text = re.compile('Twitter-Profil'))
    twitter_list.append(
        {
        'partei': "Die Linke",
        'name': abg_soup.find('h1').text,
        'twitter': twitter['href'] if twitter is not None else ""
            }
        )

In [79]:
twitter_df = pd.DataFrame(twitter_list)
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.lstrip('http://twitter.com/'))
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.lstrip('https://twitter.com/'))
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.strip(''))